In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.applications import MobileNetV2, MobileNetV3Small
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Dropout, Dense, Flatten, LSTM, TimeDistributed, GlobalAveragePooling2D, Input
from TimeDistributedImageDataGenerator.TimeDistributedImageDataGenerator import TimeDistributedImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
BATCH_SIZE = 32
IMG_HEIGHT, IMG_WIDTH = 224, 224
SEQUENCE = 10
DATA_DIRECTORY = './img/'

In [ ]:
datagen = TimeDistributedImageDataGenerator(validation_split=0.2, time_steps=SEQUENCE, preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

train_generator = datagen.flow_from_directory(
    directory=DATA_DIRECTORY, 
    batch_size=BATCH_SIZE, 
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    subset='training', 
    class_mode='sparse',
    shuffle=True
    )
    
val_generator = datagen.flow_from_directory(
    directory=DATA_DIRECTORY, 
    batch_size=BATCH_SIZE , 
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    subset='validation', 
    class_mode='sparse',
    shuffle=True
    )

np.shape(train_generator.next()[0])
np.shape(val_generator.next()[0])

class_names = list(train_generator.class_indices.keys())
num_classes = len(class_names)
print(class_names)

In [ ]:
inputs = Input(shape=(SEQUENCE, IMG_HEIGHT, IMG_WIDTH, 3))
cnn_base = MobileNetV2(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), weights="imagenet", include_top=False)
cnn_base.trainable = False

cnn_out = GlobalAveragePooling2D()(cnn_base.output)
cnn_out = Dropout(0.2)(cnn_out)
cnn_out = Flatten()(cnn_out)
cnn_out = Dense(64, activation="relu")(cnn_out)
cnn = tf.keras.Model(cnn_base.input, cnn_out)

encoded_frames = TimeDistributed(cnn)(inputs)
encoded_sequence = LSTM(64, dropout=0.1)(encoded_frames)
hidden_layer = Dense(64, activation='relu')(encoded_sequence)
outputs = Dense(num_classes)(hidden_layer)

model = tf.keras.Model(inputs, outputs)
model.summary()

In [ ]:
EPOCHS = 30
LEARNING_RATE = 0.0001

checkpoint_filepath = './tmp/'
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, monitor='val_accuracy', mode='max', save_best_only=True)
early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True) 

# model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.compile(optimizer=RMSprop(learning_rate=LEARNING_RATE), loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

history = model.fit(train_generator, validation_data=val_generator, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[early_stopping_monitor, model_checkpoint_callback])

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(EPOCHS)

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot((epochs_range), acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# Load Best Model Weights
model.load_weights(checkpoint_filepath)
model.save('model.h5')
with open('labels.txt', 'w') as f:
  f.write('sequence, ')
  f.write(', '.join(str(i) for i in class_names))